#Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!nvidia-smi

Wed May 20 21:07:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

#Imports

In [2]:
import os
import json
import imageio
import csv
import numpy as np
from operator import itemgetter

from keras.utils import to_categorical
from keras.models import load_model

import matplotlib.pyplot as plt
import matplotlib

from scipy.stats import mode
import time

import sys
sys.path.append("/content/drive/My Drive/tf-keras-SegNet")
from layers import MaxPoolingWithArgmax2D, MaxUnpooling2D
sys.path.append('/content/drive/My Drive')
from lasero.utils import training 

Using TensorFlow backend.


In [0]:
base_dir = "/content/drive/My Drive"

#Configurations and Parameters


In [4]:
session_name = "transfer_burjobands_lr_0.001"
session_paths = training.create_session_paths(session_name)
for i in session_paths:
    print(session_paths[i])

/content/drive/My Drive/models/transfer_burjobands_lr_0.001/transfer_burjobands_lr_0.001.h5
/content/drive/My Drive/models/transfer_burjobands_lr_0.001/history.json
/content/drive/My Drive/models/transfer_burjobands_lr_0.001/logs.csv
/content/drive/My Drive/models/transfer_burjobands_lr_0.001/config.json
/content/drive/My Drive/models/transfer_burjobands_lr_0.001/classification_report.txt


In [0]:
model_dir = os.path.join(base_dir, "models")

qc_dir = os.path.join(base_dir, "qc")
qc_chunk_dir = os.path.join(qc_dir, "manual_labels")
window_path = os.path.join(qc_dir, "chunk_windows.json")

metadata_dir = os.path.join(base_dir, "Metadata")
training_set_path=os.path.join(metadata_dir, 'train.txt')
val_set_path=os.path.join(metadata_dir, 'val.txt')
test_set_path=os.path.join(metadata_dir, 'test.txt')

model_path = os.path.join(model_dir, session_name)

best_model = "transfer_burjobands_lr_0.001_val_accuracy.h5"

results_dir = os.path.join(base_dir, "results", best_model)

test_set_path = "/content/drive/My Drive/Metadata/manual_chunks.csv"

test_set_list = training.get_image_list(test_set_path)

bands = [2,3,4]
num_classes = 2
image_shape = (512, 512, len(bands))
padding = ((0, 0), (0, 0))
batch_size = 5 
          
test_set_size = len(test_set_list)

In [0]:
def data_gen(metadata_file_path, bands, batch_size):
    image_list = np.asarray(training.get_image_list(metadata_file_path))
    np.random.seed(1)
    np.random.shuffle(image_list)

    band_normalization_map = []
    counter = 0

    total_steps = image_list.shape[0] // batch_size
    while True:
        step_start = counter * batch_size
        step_end = step_start + batch_size
        images = []
        masks = []
        for j in range(step_start, step_end):
            images.append(np.load(image_list[j, 0])[:,:,bands])
            masks.append(np.load(image_list[j, 1]))

        y = to_categorical(np.array(masks), 2)
        yield np.array(images), y.reshape((batch_size, y.shape[1] * y.shape[2], y.shape[3]))

        counter +=1

        if counter >= total_steps:
            counter = 0

#Predictor


In [7]:
tic = time.time()
print("start time: {}".format(tic))
model_name = best_model
current_model = os.path.join(model_path, model_name)
if not os.path.exists(current_model):
    print("current_model does not exist\n", current_model)
    sys.exit(1)
toc = time.time()

test_data = data_gen(test_set_path, bands, batch_size)
toc = time.time() - toc
print(f"test_data loaded in {toc:.2f} seconds\ntotal runtime: {time.time() - tic:.2f}")

print(current_model)
my_model = load_model(current_model, 
                      custom_objects={"MaxPoolingWithArgmax2D": MaxPoolingWithArgmax2D, 
                                      "MaxUnpooling2D": MaxUnpooling2D
                                      }
                      )
toc = time.time() - toc
print(f"model created in {toc:.2f} seconds\ntotal runtime: {time.time() - tic:.2f}")

probs = my_model.predict(test_data, steps = test_set_size // batch_size, verbose=1)

toc = time.time() - toc
print(f"predictions generated in {toc:.2f} seconds\ntotal runtime: {time.time() - tic:.2f}")

probs = probs.reshape((probs.shape[0], image_shape[0], image_shape[1], num_classes))
output = probs.argmax(axis=-1)
hist = np.histogram(output, bins=[0,1,2])
print(hist)

start time: 1590022953.2590735
test_data loaded in 0.00 seconds
total runtime: 0.36
/content/drive/My Drive/models/transfer_burjobands_lr_0.001/transfer_burjobands_lr_0.001_val_accuracy.h5
model created in 1590023001.92 seconds
total runtime: 48.66
6/6 [==============================] - 103s 17s/step
predictions generated in 103.55 seconds
total runtime: 152.21
(array([ 490728, 7373592]), array([0, 1, 2]))


In [8]:
test_scene_dir = "/content/drive/My Drive/test_chunks"
output_dir = "/content/drive/My Drive/output"

if not os.path.exists(os.path.join(output_dir, best_model)):
  os.mkdir(os.path.join(output_dir, best_model))

test_scene_ids = [i for i in os.listdir(test_scene_dir) if os.path.isdir(os.path.join(test_scene_dir, i))]
print(test_scene_ids)
for i in test_scene_ids:
  output_scene_dir = os.path.join(output_dir, best_model, i)
  if not os.path.exists(output_scene_dir):
    os.mkdir(output_scene_dir)
    print("output_dir made at {}".format(output_scene_dir))

['LC82201072015017LGN00', 'LC82071182013336LGN00', 'LC81431082013352LGN00', 'LC81681112014002LGN00', 'LC80631112014002LGN00', 'LC80311222014338LGN00', 'LC81041072013303LGN00', 'LC82091172014001LGN00', 'LC82131132013362LGN00']


In [9]:
for i in range(output.shape[0]):
    chunk_path, chunk_name = os.path.split(test_set_list[i][0])
    scene_chunk_path, scene_id = os.path.split(chunk_path)
    name, ext = os.path.splitext(chunk_name)
    out_file_name = f"{name}_output{ext}"
    print(os.path.join(output_dir, model_name, scene_id, out_file_name))
    np.save(os.path.join(output_dir, model_name, scene_id, out_file_name), output[i])

/content/drive/My Drive/output/transfer_burjobands_lr_0.001_val_accuracy.h5/LC80311222014338LGN00/chunk_6_1_output.npy
/content/drive/My Drive/output/transfer_burjobands_lr_0.001_val_accuracy.h5/LC80311222014338LGN00/chunk_6_2_output.npy
/content/drive/My Drive/output/transfer_burjobands_lr_0.001_val_accuracy.h5/LC80311222014338LGN00/chunk_7_1_output.npy
/content/drive/My Drive/output/transfer_burjobands_lr_0.001_val_accuracy.h5/LC80311222014338LGN00/chunk_7_2_output.npy
/content/drive/My Drive/output/transfer_burjobands_lr_0.001_val_accuracy.h5/LC80631112014002LGN00/chunk_6_6_output.npy
/content/drive/My Drive/output/transfer_burjobands_lr_0.001_val_accuracy.h5/LC80631112014002LGN00/chunk_6_7_output.npy
/content/drive/My Drive/output/transfer_burjobands_lr_0.001_val_accuracy.h5/LC81041072013303LGN00/chunk_6_14_output.npy
/content/drive/My Drive/output/transfer_burjobands_lr_0.001_val_accuracy.h5/LC81041072013303LGN00/chunk_7_14_output.npy
/content/drive/My Drive/output/transfer_burjob